In [1]:
from __future__ import print_function
import numpy as np
import statsmodels.api as sm
import pandas as pd

In [2]:
import numpy as np
import statsmodels.api as sm
np.random.seed(12345)
arparams = np.array([.75, -.25])
maparams = np.array([.65, .35])
ar = np.r_[1, -arparams] # add zero-lag and negate
ma = np.r_[1, maparams] # add zero-lag
y = sm.tsa.arma_generate_sample(ar, ma, 250)
model = sm.tsa.ARMA(y, (2, 2)).fit(trend='nc', disp=0)
model.params

array([ 0.79044192, -0.23140636,  0.70072906,  0.40608023])